<a href="https://colab.research.google.com/github/SalvatoreDiliberto/Dataset/blob/main/Esercizio%20superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6e95faf406fd48ecb1ff3cb16b93b1731ce0cea1e3c26ff300a3525bb228af65
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()


In [3]:
spark

In [7]:
# read our data - lives in a csv file

df = spark.read.option("header","true").csv("/content/sample_data/Sample - EU Superstore.csv")

In [17]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df['Country'] == 'France').count()


2827

In [30]:
# of those, how many are profitable?
df.filter((df['Country'] == 'France') & (df['Profit'] > 0)).count()

2277

In [23]:
# how any different discount backets exist? what are they?
df.select('Discount').distinct().orderBy('Discount', ascending=False).show()

+--------+
|Discount|
+--------+
|    0.85|
|     0.8|
|     0.7|
|    0.65|
|     0.6|
|     0.5|
|    0.45|
|     0.4|
|    0.35|
|     0.3|
|     0.2|
|    0.15|
|     0.1|
|       0|
+--------+



In [32]:
# let's see the totl profit by discount backet, make sure they are ordered by
df2 = df.groupBy('Discount').agg({'Profit' : 'sum'}).orderBy('Discount', ascending=False)

In [35]:
df2.show()

+--------+-------------------+
|Discount|        sum(Profit)|
+--------+-------------------+
|    0.85|          -3068.658|
|     0.8|           -460.284|
|     0.7|          -5496.765|
|    0.65| -6221.965499999999|
|     0.6|-20517.456000000002|
|     0.5|         -96632.115|
|    0.45|         -1103.1915|
|     0.4|-21346.427999999996|
|    0.35|          -9122.649|
|     0.3| -758.4209999999999|
|     0.2| 2189.5499999999984|
|    0.15| 24677.563499999975|
|     0.1|  126884.0309999999|
|       0| 383806.53000000026|
+--------+-------------------+



In [47]:
# what is the value after which we should stop offering discount?
df3 = df2.filter(df2['sum(Profit)'] > 0)
df3.show()

+--------+------------------+
|Discount|       sum(Profit)|
+--------+------------------+
|     0.2|2189.5499999999984|
|    0.15|24677.563499999975|
|     0.1| 126884.0309999999|
|       0|383806.53000000026|
+--------+------------------+



In [48]:
df3.select('Discount').show(1)

+--------+
|Discount|
+--------+
|     0.2|
+--------+
only showing top 1 row



In [101]:
# who are the top 5 most profitable customers
df4 = df.groupBy('Customer Name').agg({'Profit':'sum'}).orderBy('sum(Profit)', ascending=False)
df4.show(5)


+-----------------+------------------+
|    Customer Name|       sum(Profit)|
+-----------------+------------------+
|     Susan Pistek| 4974.512999999999|
|    Patrick Jones|3986.0039999999995|
|Patrick O'Donnell|          3778.197|
|    Ellis Ballard|           3459.66|
|  Mike Gockenbach|3144.4439999999995|
+-----------------+------------------+
only showing top 5 rows



In [102]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are th
top5_rows = df4.collect()
top5_customer_names = [row['Customer Name'] for row in top5_rows]
rows_for_top5_customers = df.filter(df['Customer Name'].isin(top5_customer_names))
print({rows_for_top5_customers.count()})



{10000}


In [78]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)
df5 = df.withColumn('Vendita', df['Sales'] / (1 - df['Discount']))
df5.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|           Vendita|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corporate|        Leeds|             England|United Kingdom|  Nor

In [84]:
# calculate the difference between sales and discount value
df5 = df5.withColumn('Differenza', df5['Sales'] - df5['Vendita'])
df5.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|           Vendita|         Differenza|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corpo

In [88]:
# how much money did we not gain due to the discounts - per discount bracket?
df5.select('Profit','Vendita','Differenza').show()


+-------+------------------+-------------------+
| Profit|           Vendita|         Differenza|
+-------+------------------+-------------------+
|   39.6|              79.2|                0.0|
|      0|            388.92|                0.0|
|  16.11|             35.19|                0.0|
|   13.2|             50.94|                0.0|
|  73.71|            307.44|                0.0|
|  37.92|             122.4|                0.0|
|  20.61|            413.82|                0.0|
| 192.69|            428.22|                0.0|
|1989.54|           3979.29|                0.0|
|   12.6|             43.56|                0.0|
|   0.48|             25.26|                0.0|
|760.305|2715.4500000000003| -271.5450000000001|
|      0|             12.21|                0.0|
|790.416|2549.7599999999998|-382.46399999999994|
|-12.345|            153.45|-15.344999999999999|
|  4.275|142.64999999999998|-14.264999999999986|
|      0|            690.12|                0.0|
|   1.14|           

In [89]:
df5.groupBy('Discount').agg({'Differenza' : 'sum'}).orderBy('Discount', ascending=False)
df5.show()

+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+
|Row ID|       Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|         City|               State|       Country| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount| Profit|           Vendita|         Differenza|
+------+---------------+----------+----------+--------------+-----------+----------------+-----------+-------------+--------------------+--------------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+-------+------------------+-------------------+
|     1|ES-2017-1311038|07/02/2017|11/02/2017|Standard Class|   AS-10045|  Aaron Smayling|  Corpo

In [114]:
# find the discount bracket which made us not gain the most (dynamically)
df2 = df.groupBy('Discount').agg({'Profit' : 'sum'}).orderBy('Discount', ascending=False)
df2.show()

+--------+-------------------+
|Discount|        sum(Profit)|
+--------+-------------------+
|    0.85|          -3068.658|
|     0.8|           -460.284|
|     0.7|          -5496.765|
|    0.65| -6221.965499999999|
|     0.6|-20517.456000000002|
|     0.5|         -96632.115|
|    0.45|         -1103.1915|
|     0.4|-21346.427999999996|
|    0.35|          -9122.649|
|     0.3| -758.4209999999999|
|     0.2| 2189.5499999999984|
|    0.15| 24677.563499999975|
|     0.1|  126884.0309999999|
|       0| 383806.53000000026|
+--------+-------------------+



In [122]:
from pyspark.sql.functions import min
df2.select(min('sum(Profit)')).show()


+----------------+
|min(sum(Profit))|
+----------------+
|      -96632.115|
+----------------+



In [133]:
# what would have been the total profit if we removed all orders from that discount group?
# quale sarebbe stato il profitto totale se avessimo rimosso tutti gli ordini da quel gruppo di sconto?
from pyspark.sql.functions import sum
a = df2.filter(df2['Discount']!= 0.5).agg(sum(df2['sum(Profit)'])).first()[0]


In [134]:
#how much more (or less) profit is that?
#quanto profitto in più (o in meno) è?
from pyspark.sql.functions import sum
b = df2.agg(sum(df2['sum(Profit)'])).first()[0]


In [136]:
df_differenza = a - b
df_differenza

96632.11499999999

In [105]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView('sales')

In [109]:
# use an SQL query to count the number of rows
spark.sql('Select count(*) From sales').show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [139]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)
# Utilizza una query SQL per calcolare il rapporto di profitto per ciascun paese: suggerimento, il rapporto è somma(profitto)/somma(vendite)
spark.sql('Select sum(Profit)/sum(Sales), Country from sales group by Country').show()


+--------------------------+--------------+
|(sum(Profit) / sum(Sales))|       Country|
+--------------------------+--------------+
|       -0.5745674280714466|        Sweden|
|       0.17066792076621765|       Germany|
|       0.12693568221933804|        France|
|       0.23508766583987942|       Belgium|
|       0.18864296633316185|       Finland|
|       0.06844355185424991|         Italy|
|        0.2517747548521659|        Norway|
|       0.18941580658358978|         Spain|
|       -0.4957190005664471|       Denmark|
|      -0.44426677493909256|       Ireland|
|        0.2909201193350232|   Switzerland|
|       -0.5761662270806188|      Portugal|
|        0.2641908775042505|       Austria|
|       0.21170103540397134|United Kingdom|
|       -0.5298342790541865|   Netherlands|
+--------------------------+--------------+



In [142]:
# is the country with the largest profit ratio, the country with the largest profit?
# il paese con il rapporto di profitto maggiore è il paese con il profitto maggiore?
spark.sql('Select (sum(Profit) / sum(Sales)) from sales ')


DataFrame[(sum(Profit) / sum(Sales)): double]